In [1]:
import datasets
from typing import List, Dict, Union, Literal, Optional
from tqdm import tqdm

In [2]:
dataset = datasets.load_dataset("blindsubmissions/GH_text2code", split="python_gh")

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/73 [00:00<?, ?it/s]

In [3]:
for item in dataset:
    print(item['function'])
    break

async def async_setup_entry(hass, entry, async_add_entities):
    """Set up SimpliSafe locks based on a config entry."""
    simplisafe = hass.data[DOMAIN][DATA_CLIENT][entry.entry_id]
    async_add_entities(
        [
            SimpliSafeLock(simplisafe, system, lock)
            for system in simplisafe.systems.values()
            for lock in system.locks.values()
        ]
    )


In [4]:
# # Задаем количество строк, которое хотим получить
# n_rows = 2000

# start_row = 0
# end_row = 2000

# # Создаем список для хранения строк
# rows = []

# for i, row in enumerate(dataset):
#     if start_row <= i < end_row:
#         rows.append(row)
#     elif i >= end_row:
#         break

# # Преобразуем список строк в датасет Hugging Face
# dataset_w_n_samples = datasets.Dataset.from_dict({k: [row[k] for row in rows] for k in rows[0]})

In [5]:
def format_class(class_name, methods):
    class_definition = f"class {class_name}:\n"
    for method in methods:
        # Split the method into lines and add the indentation
        lines = method.split('\n')
        # Indent all lines except the first one with four spaces
        indented_lines = ['    ' + line.replace("        ", "    ") for line in lines]
        # Join the lines and add them to the class definition
        class_definition += '\n'.join(indented_lines) + '\n\n'
    return class_definition

In [ ]:
from tqdm import tqdm

def split_identifier(identifier: str) -> Union[str, None]:
    splitted = identifier.split('.')
    if len(splitted) > 1:
        return splitted[0]
    elif len(splitted) == 1:
        return None

def process_dataset(dataset: List[Dict[str, str]]) -> Dict[str, Dict[Literal["class", "func"], List[str]]]:
    res_dict: Dict[Literal["class", "func"], Dict[str, List[str]]] = {"class": {}, "func": []}

    for item in tqdm(dataset):
        class_name = split_identifier(item['identifier'])
        if class_name:
            if class_name not in res_dict["class"]:
                res_dict["class"][class_name] = [item["function"]]
            else:
                res_dict["class"][class_name].append(item["function"])
        else:
            res_dict["func"].append(item["function"])

    return res_dict

def format_classes(classes: Dict[str, List[str]]) -> List[str]:
    formatted_classes = []
    for class_name, functions in classes.items():
        class_definition = format_class(class_name, functions)
        formatted_classes.append(class_definition)
    return formatted_classes

# Использование функций
res_dict = process_dataset(dataset)
res_dict["class"] = format_classes(res_dict["class"])

 43%|████▎     | 6450145/15000002 [20:13<29:01, 4909.85it/s]  

In [7]:
print(res_dict["class"][0])

class SimpliSafeLock:
    def __init__(self, simplisafe, system, lock):
        """Initialize."""
        super().__init__(simplisafe, system, lock.name, serial=lock.serial)
        self._lock = lock
        self._is_locked = None
    
        for event_type in (EVENT_LOCK_LOCKED, EVENT_LOCK_UNLOCKED):
            self.websocket_events_to_listen_for.append(event_type)

    def is_locked(self):
        """Return true if the lock is locked."""
        return self._is_locked

    async def async_lock(self, **kwargs):
        """Lock the lock."""
        try:
            await self._lock.lock()
        except SimplipyError as err:
            LOGGER.error('Error while locking "%s": %s', self._lock.name, err)
            return

    async def async_unlock(self, **kwargs):
        """Unlock the lock."""
        try:
            await self._lock.unlock()
        except SimplipyError as err:
            LOGGER.error('Error while unlocking "%s": %s', self._lock.name, err)
            return

  